Install Pygame

In [2]:
%pip install pygame

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 1.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


#### -- [INIT] -- 
***

##### **SETUP**
- `on_awake   ` (init_game => screen, refresh, load assets ... etc)
- `on_update  ` (game_loop => handle events, input, draws ..etc )
- `on_exit    ` (close_app => clear memory and shut down cpu/sys ... etc)

In [100]:
# `numpy` library random which is FASTER for generating large numbers of random values
from numpy import random
import pygame
import sys

SCREEN_WIDTH, SCREEN_HEIGHT = 320, 240

def on_awake():
    ''' 
    Initialize pygame and return:
    - screen (pygame.Surface)
    - clock (pygame.time.Clock)
    '''
    # Initialize pygame
    pygame.init()

    # Define the screen size
    # Create the screen
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

    # @audit-ok 💨 : Cap the frame rate else loop will run at system allows and 
    # will lead to :
    # - high CPU usage
    # - inconsistent performance across different devices. 
    clock = pygame.time.Clock()

    # Set the title and icon (optional)
    pygame.display.set_caption('Asteroids_TON_Edition')
    icon = pygame.image.load('asteroid.png')
    pygame.display.set_icon(icon)
    return screen, clock

def on_exit():
    ''' 
    Handle exit event
    '''
    pygame.quit()
    sys.exit()

# Game Loop
def on_update(screen : pygame.Surface, clock : pygame.time.Clock, asteroids, FPS=30):
    ''' 
    Game loop
    '''
    # Set the frame rate
    while True:
        try:
            clock.tick(FPS)
            # Event Handling
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    on_exit()
                if event.type == pygame.KEYUP:
                    if event.key == pygame.K_ESCAPE:
                        on_exit()
            # Update game state
            # Draw game objects
            # Update the display
            r = random.randint(0, 255)
            g = random.randint(0, 255)
            b = random.randint(0, 255)
            screen.fill((r, g, b))
            for asteroid in asteroids:
                asteroid.update()
                asteroid.draw(screen)
            pygame.display.flip()
            
        except SystemExit:
            break

##### **CLASSES**

- Asteroids

In [101]:
class Asteroid:
    def __init__(self, x, y, image):
        self.x = x
        self.y = y
        self.image = image
        self.vx = random.randint(-2, 2)
        self.vy = random.randint(-2, 2)
        self.rotation = random.randint(0, 359)

    def update(self):
        # Update the position based on velocity
        # self.x += self.vx
        # self.y += self.vy
        # Update the rotation
        self.rotation = (self.rotation + 1) % 360

    def draw(self, screen):
        # Rotate the image
        rotated_image = pygame.transform.rotate(self.image, self.rotation)
        # Calculate the new image's rect to keep it centered
        rect = self.image.get_rect(center=self.image.get_rect(topleft=(self.x, self.y)).center)
        # Draw the asteroid
        screen.blit(rotated_image, rect.topleft)

#### -- [ON_AWAKE] --

In [108]:
screen, clock = on_awake()
asteroids = []
x = 160
y = 120
img = pygame.image.load('asteroid.png')
img = pygame.transform.scale(img, (32, 32))
asteroid = Asteroid(x, y, img)
asteroids.append(asteroid)

#### -- [ON_UPDATE] --

In [109]:
# @audit-ok 💨 : The memory and performance overhead of passing the `screen` 
# object to the function is MINIMAL, as it's just passing a REFERENCE to the 
# object.
FPS = 60
on_update(screen, clock, asteroids, FPS)

#### -- [FOOTNOTES] --
***

Explain the `difference` between adding the `try and exception catching` above vs  
not doing a catch from `memory` and `performance` perspective
***

Adding a `try` and `except` block around your code introduces a minimal overhead  
in terms of memory and performance.  In general, the cost of using exception  
handling is low when no exceptions are raised.  

However, when an exception occurs : 
- the traceback needs to be generated  
- and the `interpreter has to search the exception hierarchy` to find the  
appropriate `except` block, which can be more `expensive`.

Here's a comparison between using exception handling and not using it in the  
context of the Pygame example we've discussed:

**Without exception handling:**

When you don't catch the `SystemExit` exception, the kernel running the Jupyter  
Notebook will display the message : 
- `An exception has occurred, use %tb to see the full traceback. SystemExit`  
whenever the game is terminated. 

This is because : 

- the `SystemExit` exception is raised, 
- caught by the notebook's kernel, 
- and displayed as output. 

The game loop will still exit as expected, and the exception handling overhead  
is minimal. However, you'll see the traceback message every time the game is  
closed, which might be `undesirable from a user experience standpoint`.

**With exception handling:**

When you include a `try` and `except` block to catch the `SystemExit` exception,  
you introduce a small overhead associated with the exception handling mechanism.  

However, this overhead is generally negligible in the context of a game loop.  
When the game is terminated, the `SystemExit` exception is raised and caught  
by the `except` block, allowing the loop to exit silently without displaying  
the traceback message. This can provide a cleaner user experience.

In summary, the difference in memory and performance between using exception  
handling and not using it in this case is minimal. The primary reason to include  
exception handling in this scenario is to improve the user experience by  
preventing the traceback message from being displayed when the game is terminated.